## Preprocesamiento de datos

Este notebook muestra como limpiar y normalizar los valores de cuota de las AFP en Chile. Los datos de los fondos de pensiones fueron descargados desde el sitio web de la [Superintendencia de Pensiones](https://www.spensiones.cl/apps/valoresCuotaFondo/vcfAFP.php)

In [29]:
import pandas as pd
import numpy as np
import os
import re
import requests

from datetime import datetime

## Descargar los datos actualizados desde el sitio web

In [30]:
dataDir = '../data'
today = datetime.today()

dataFileDF = pd.DataFrame([{'fondo':'A', 'URL':'https://www.spensiones.cl/apps/valoresCuotaFondo/vcfAFPxls.php?tf={}&aaaaini=0000&aaaafin={}&fecconf={}'.format('A', today.strftime("%Y"), today.strftime("%Y%m%d"))},
                          {'fondo':'B', 'URL':'https://www.spensiones.cl/apps/valoresCuotaFondo/vcfAFPxls.php?tf={}&aaaaini=0000&aaaafin={}&fecconf={}'.format('B', today.strftime("%Y"), today.strftime("%Y%m%d"))},
                          {'fondo':'C', 'URL':'https://www.spensiones.cl/apps/valoresCuotaFondo/vcfAFPxls.php?tf={}&aaaaini=0000&aaaafin={}&fecconf={}'.format('C', today.strftime("%Y"), today.strftime("%Y%m%d"))},
                          {'fondo':'D', 'URL':'https://www.spensiones.cl/apps/valoresCuotaFondo/vcfAFPxls.php?tf={}&aaaaini=0000&aaaafin={}&fecconf={}'.format('D', today.strftime("%Y"), today.strftime("%Y%m%d"))},
                          {'fondo':'E', 'URL':'https://www.spensiones.cl/apps/valoresCuotaFondo/vcfAFPxls.php?tf={}&aaaaini=0000&aaaafin={}&fecconf={}'.format('E', today.strftime("%Y"), today.strftime("%Y%m%d"))}])

dataFileDF.head()

,fondo,URL
0,A,https://www.spensiones.cl/apps/valoresCuotaFon...
1,B,https://www.spensiones.cl/apps/valoresCuotaFon...
2,C,https://www.spensiones.cl/apps/valoresCuotaFon...
3,D,https://www.spensiones.cl/apps/valoresCuotaFon...
4,E,https://www.spensiones.cl/apps/valoresCuotaFon...


In [31]:
for index, rowFile in dataFileDF.iterrows():
    
    url = rowFile['URL']
    print("Downloading URL: ", url)
    r = requests.get(url)
    if r.status_code == 200:
        content = r.headers['content-disposition']
        
        fileName = re.findall("filename=(.+\.\w{3})", content)[0]
        fileName = os.path.join(dataDir, fileName)
        
        with open(fileName, 'wb') as f:
            f.write(r.content)
            dataFileDF.loc[index, 'fileName'] = fileName

## Creamos la carpeta results para guardar el output

In [32]:
if not os.path.exists('../results'):
    os.mkdir('../results')

In [33]:
reHeader = re.compile(r'^Fecha.*')
reData = re.compile(r'^\d{4}-\d{2}-\d{2}.*')
       
dataList=[]
for index, rowFile in dataFileDF.iterrows():
    
    print("Processing file: ", rowFile['fileName'])
    
    dataRaw = []
    with open(rowFile['fileName'], 'r') as f:
        for line in f:
            #print(line)
            matchHeader = reHeader.match(line)
            matchData = reData.match(line)
            if matchHeader:
                headerRaw = matchHeader.group(0)
            if matchData:
                dataRaw.append(matchData.group(0))
                
    header = headerRaw.split(';')
    
    for rowData in dataRaw:
        rowValues = rowData.split(';')
                
        for i in range(1, len(header), 2):
            dataDict={'fecha': datetime.strptime(rowValues[0], '%Y-%m-%d'),
                      'fondo':rowFile['fondo'] }

            dataDict['administradora'] = header[i]
            if re.match("[0-9]+\..*", rowValues[i]):
                dataDict['valor cuota'] = float(rowValues[i].replace('.','').replace(',','.'))
            else:
                dataDict['valor cuota'] = np.nan
            if re.match("[0-9]+.*", rowValues[i]):
                dataDict['valor patrimonio'] = float(rowValues[i+1])
            else:
                dataDict['valor cuota'] = np.nan
            
            dataList.append(dataDict)
           
dataDF = pd.DataFrame(dataList)

Processing file:  ../data/vcfA2020-2020.csv
Processing file:  ../data/vcfB2020-2020.csv
Processing file:  ../data/vcfC2020-2020.csv
Processing file:  ../data/vcfD2020-2020.csv
Processing file:  ../data/vcfE2020-2020.csv


In [34]:
dataDF[dataDF['fondo']=='A'].tail(7)

,fecha,fondo,administradora,valor cuota,valor patrimonio
1078,2020-05-17,A,CAPITAL,42782.52,3.382642e+12
1079,2020-05-17,A,CUPRUM,44737.67,4.005806e+12
1080,2020-05-17,A,HABITAT,45825.88,5.155138e+12
1081,2020-05-17,A,MODELO,43880.69,1.098766e+12
1082,2020-05-17,A,PLANVITAL,41741.57,4.395276e+11
1083,2020-05-17,A,PROVIDA,45035.18,3.388587e+12
1084,2020-05-17,A,UNO,NaN,NaN


In [35]:
outputFileName = '../results/afp_chile_2020.csv'

dataDF.to_csv(outputFileName, index=False)